# [ DataMining ] 지도학습 - 앙상블 학습

## * 앙상블 학습

#### - 트레이닝 데이터를 기반으로 분류 모형을 여러개 만들고 서로 비교하는 것
#### - 여러 개의 분류기를 결합함으로써 개별적인 분류기보다 성능이 뛰어난 최종 분류기를 만드는 것이 앙상블 학습의 목적

---

## 1. 보팅

- 여러 개의 분류 모형의 결과를 대상으로 투표를 통해 최종 클래스 라벨을 정하는 방법
- 다수결 투표 / 과반수 투표
- 앙상블 학습 모형은 개별 모형보다 성능이 뛰어남
- 모든 개별 분류기의 오차는 서로 독립적이라서 서로 상관관계가 없다고 가정
    즉, n개의 분류기 중 k개 이상의 분류기의 예측이 틀렸다면 $$P(X \geq k) = \sum_{k=i}^N {n \choose k} x^k (1-x)^{n-k} = \varepsilon _{ensenble}$$ 
- X가 k보다 크거나 같은 이유: 앙상블 분류기가 틀리려면 과반수의 개별 분류기가 틀려야 함

In [1]:
'''0. 데이터 불러오기'''

from sklearn import datasets
iris = datasets.load_iris()

In [3]:
'''1. feature, target data 지정'''

x = iris.data
y = iris.target

In [4]:
'''2. 트레이닝/테스트 데이터 분할'''

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=0)

In [6]:
'''3. 데이터 표준화'''

from sklearn.preprocessing import StandardScaler

std = StandardScaler()      # 표준화 스케일러를 지정
std.fit(x_train)            # training feature를 기준으로 표준화를 적합

x_train_std = std.transform(x_train)  # training data를 fit시킨 표준화에 맞게 변형
x_test_std = std.transform(x_test)    # test data를 fit시킨 표준화에 맞게 변형

In [9]:
'''4. 데이터 학습 - 보팅학습'''

from sklearn.linear_model import LogisticRegression  
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression(multi_class='multinomial', random_state=1)  # 로지스틱 회귀
svm_clf = SVC(kernel='linear', random_state=1)                           # 서포트 벡터 머신
gnb_clf = GaussianNB()                                                   # 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators=[('lr',log_clf),('svm',svm_clf),('gnb',gnb_clf)],
                             voting='hard', weights=[1,1,1])             # estimators=위의 세가지 모형
# voting option: hard(투표결과로 과반수가 넘는 라벨, 기본값), soft(확률이 높은 라벨)

clf_voting.fit(x_train_std, y_train)  # 만들어진 모델에 fit(적합)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

In [12]:
'''5. 데이터 예측 (pred_voting = y_hat)'''

pred_voting = clf_voting.predict(x_test_std)
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


In [13]:
'''6. 정확도 평가'''

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred_voting)
print(accuracy)

0.9736842105263158


In [16]:
'''7. confusion matrix 확인'''

from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, pred_voting)
print(conf_mat)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


In [17]:
'''8. 분류 리포트 확인'''

from sklearn.metrics import classification_report
class_rpt = classification_report(y_test, pred_voting)
print(class_rpt)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



---

## 2. 배깅과 랜덤 포레스트 - 독립적 앙상블 방법

